# Анализ базы данных сервиса для чтения книг по подписке

**Цель:**

Провести анализ базы данных крупного сервиса для чтения книг по подписке для последующего формулирования ценностного предложения для нового продукта.


### Подключение библиотек

Подключим необходимые для работы библиотеки.

In [1]:
import pandas as pd
import sqlalchemy as sa

### Подключение к базе данных

Установим необходимые параметры и напишем функцию для выполнения SQL-запроса.

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

### Обзор базы данных

Проведем обзор таблиц базы данных. Для удобства напишем две функции: одна будет выводить первые строки таблицы, вторая - количество строк в таблице.

In [3]:
#Выведем первые 5 строк таблицы
def first_lines_table(table, table_id):
    query = '''
    SELECT * 
    FROM %s 
    ORDER BY %s ASC 
    LIMIT 5;''' % (table, table_id)
    return get_sql_data(query).style.set_caption('Таблица %s' % (table))

In [4]:
#Выведем кол-во строк таблицы
def col_number(table):
    query = '''
    SELECT count(*) 
    FROM %s;''' % (table)
    return get_sql_data(query).style.set_caption('Кол-во строк в %s' % (table))

In [5]:
display(first_lines_table("books", "book_id"), col_number("books"))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,count
0,1000


In [6]:
display(first_lines_table("authors", "author_id"), col_number("authors"))

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,count
0,636


In [7]:
display(first_lines_table("ratings", "rating_id"), col_number("ratings"))

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,count
0,6456


In [8]:
display(first_lines_table("reviews", "review_id"), col_number("reviews"))

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


,count
0,2793


In [9]:
display(first_lines_table("publishers", "publisher_id"), col_number("publishers"))

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,count
0,340


**Вывод:**

Таблицы базы данных соответствуют описаниям и обозначенной структуре. Количество столбцов и данные, содержащиеся в таблицах также соответсвуют. 

При изучении таблиц было выяснено, что в приложении сервиса содержится 1000 книг, написанных 636 авторами.

### Задания

**Посчитаем, сколько книг вышло после 1 января 2000 года**

In [10]:
query = '''
SELECT count(book_id) 
FROM books 
WHERE publication_date > '2000-01-01';'''
get_sql_data(query).style.set_caption('Кол-во книг')

,count
0,819


После 1 января 2000 года вышло 819 книг и брошюр.

**Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [14]:
query = '''
SELECT b.book_id,
b.title,
count( DISTINCT r.review_id) AS cnt_reviews,
round(avg(r2.rating), 2) AS avg_rating
FROM books b 
LEFT JOIN reviews r ON  b.book_id = r.book_id 
LEFT JOIN ratings r2 ON b.book_id = r2.book_id 
GROUP BY b.book_id
ORDER BY cnt_reviews DESC;'''
get_sql_data(query).sample(5).style.set_caption('Кол-во обзоров и средняя оценка книг')

,book_id,title,cnt_reviews,avg_rating
15,497,Outlander (Outlander #1),6,4.130000
544,77,Angels Flight (Harry Bosch #6; Harry Bosch Universe #7),2,4.500000
413,764,The Island of Doctor Moreau,3,4.000000
507,956,Vanity Fair,3,4.000000
700,398,Little Town on the Prairie (Little House #7),2,4.330000


При изучении таблицы "books" мы выяснили, что в ней содержится 1000 книг. Количество обзоров на книги варируется от 7 до 0. Также можно заметить, что средний рейтинг не зависит от количества обзоров на книгу.


**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц - так исключим из анализа брошюры**

In [15]:
query = '''
SELECT p.publisher_id,
p.publisher,
count(b.book_id) AS amount_of_books
FROM publishers p 
INNER JOIN books b ON p.publisher_id = b.publisher_id 
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY count(b.book_id) DESC 
LIMIT 1;'''
get_sql_data(query).style.set_caption('Издательство выпустившее наибольшее число книг')

,publisher_id,publisher,amount_of_books
0,212,Penguin Books,42


Издательство, выпустившее наибольшее количество книг толще 50 страниц - британское издательство "Penguin Books". Количество выпущенных книг равняется 42. 

**Определим автора с самой высокой средней оценкой книг - учитываем только книги с 50 и более оценками**

In [16]:
#Сначала отберем книгис оценками 50 и более, 
#а потом определим автора с наивысшей средней оценкой книг
query = '''
WITH top_books AS (
    SELECT b.author_id,   
    round(avg(r.rating), 2) AS avg_rating
    FROM books b
    INNER JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.book_id 
    HAVING count(r.rating_id) >= 50
    ORDER BY b.author_id
)
SELECT a.author_id,
a.author,
avg(tb.avg_rating) AS overall_avg_rating
FROM authors a
INNER JOIN top_books tb ON a.author_id = tb.author_id
GROUP BY a.author_id
ORDER BY avg(tb.avg_rating) DESC
LIMIT 1
'''
get_sql_data(query).style.set_caption('Автор с самой высокой средней оценкой книг')

,author_id,author,overall_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.285000


Автором с самой высокой средней оценкой книг **4.285** является британская писательница **Джоан Роулинг**. Она известна своей серией книг и Гарри Поттере. В тандеме с ней находится американская иллюстратор **Мэри Гранпрэ**, наиболее известная своими иллюстрациями обложек и глав книг о Гарри Поттере в их американских изданиях, опубликованных Scholastic.

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок**

In [17]:
query = '''
SELECT count(DISTINCT r2.review_id) / count(DISTINCT r2.username) AS avg_reviews
FROM ratings r
INNER JOIN reviews r2 ON r.username = r2.username
WHERE r.username in ( 
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48
                    );'''
get_sql_data(query).style.set_caption('Среднее кол-во обзоров')

,avg_reviews
0,24


В среднем пользователей, которые поставили больше 48 оценок книгам, написали **24 обзора** на книги. 

### Вывод:

В данной работе был произведен анализ крупного сервиса для чтения книг по подписке. Данные в предоставленных таблицах соответствовали заявленым. При проведении анализа было выяснено, что в приложении содержится 1000 книг от 636 авторов. При этом 819 книг и брошюр было выпущено после 2000 года.

Было выяснено то, что книги имеют рейтинг и обзоры (обзоры есть не на все из них), а также, что рейтинг книги не зависит от количества обзоров на нее, так как рейтинг книги отражает оценку пользователей разных вкусовых предпочтений.

Из всех издательств представленных в базе самым популярным является издательство "Pinguin books", оно выпустило наибольшее количество книг. А самым популярным автором с рейтингом 4.29 является Джоан Роулинг с ее серией книг о Гарри Поттере.

Также было выяснено, что наиболее активные пользователи приложения, поставившие больше 48 оценок книгам, в среднем написали 24 обзора на книги.

**Рекомендации:**

На основании вышеизложенного, для успеха приложения по чтению книг, можно расширить базу книжными новинками. Поскольку популярны книги Джоан Роулинг, можно добавить в приложение ее книги на разных языках, например в оригинале или добавить книги о том, как создавалась ее популярная серия книг. 

Также можно добавить больше книг от издательства Pinguin books. Так как это издательство известно тем, что сделало книги доступными для большого числа людей, а не оставило книги роскошью. Поэтому у них можно найти книги по доступным для читателей ценам. 

Можно добавить дополнительный рейтинг книгам. Сейчас рейтинг формируется на основании оценок пользователей, а вкусы у пользователей могут быть разными, сложно дать объективную оценку, поэтому можно добавить к популярным книгам рейтинги критиков или добавить плашку что книга входит, например, в "Топ-100" лучших или популярных книг.